# Get Traffic Data Bandung

In [1]:
import geopandas as gpd

# Read the shapefile
shapefile_path = "../data/shapefile/centroid_bandung.shp"
gdf = gpd.read_file(shapefile_path)

In [2]:
# Extract coordinates and attribute data
gdf['latitude'] = gdf.geometry.y
gdf['longitude'] = gdf.geometry.x

In [3]:
gdf =  gdf[['NAME_4', 'latitude', 'longitude']]

In [15]:
centroid = gdf.rename(columns = {
    'NAME_4':'kelurahan'
})
centroid

,kelurahan,latitude,longitude
0,Campaka,-6.900412,107.566504
1,Ciroyom,-6.915770,107.588252
2,Dungus Cariang,-6.914119,107.580806
3,Garuda,-6.911455,107.576032
4,Kebon Jeruk,-6.917187,107.598898
...,...,...,...
146,Cigending,-6.910509,107.697963
147,Pasanggrahan,-6.910776,107.713816
148,Pasir Endah,-6.904375,107.690213
149,Pasir Wangi,-6.899396,107.706198


### Get Traffic Matrix
Setiap peak hour? -> mau di tanggal berapa?
Cek dulu peak hour